In [1]:
import tensorflow as tf
import numpy as np


C:\Users\young hwa\.conda\envs\heawon\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\young hwa\.conda\envs\heawon\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\young hwa\.conda\envs\heawon\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\young hwa\.conda\envs\heawon\lib\site-packages\tensorflow\

In [2]:
import math

DataList=[]
p=0
n=0
m1=0
m2=20000
for i in range(m1,m2):
    k=int(math.sqrt(i)//1)+1
    if i==0 or i==1:
        n=1
    for j in range(2,k):
        if i%j==0:
            n+=1
            break
    if n==0:
        p=1
    DataList.append([i,p])
    p=0
    n=0
    

DataList0=[]
p=0
n=0
m3=20000
m4=24000
for i in range(m3,m4):
    k=int(math.sqrt(i)//1)+1
    for j in range(2,k):
        if i%j==0:
            n+=1
            break
    if n==0:
        p=1
    DataList0.append([i,p])
    p=0
    n=0
    
DataList=np.transpose(DataList)
DataList0=np.transpose(DataList0)

In [3]:
InputDataLIst=[]
list1=[]
for i in range( len(DataList[0]) ):
    for j in range(784):
        list1.append(DataList[0][i])
    InputDataLIst.append(list1)
    list1=[]
    
InputDataLIst0=[]
list1=[]
for i in range( len(DataList0[0]) ):
    for j in range(784):
        list1.append(DataList0[0][i])
    InputDataLIst0.append(list1)
    list1=[]
    
    

In [4]:

x_train=np.array(InputDataLIst)
y_train=np.array(DataList[1])
x_test=np.array(InputDataLIst0)
y_test=np.array(DataList0[1])


In [5]:
x_val  = x_train[int(m2*8/10):m2]
x_train = x_train[0:int(m2*8/10)]
y_val  = y_train[int(m2*8/10):m2]
y_train = y_train[0:int(m2*8/10)]

In [6]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 2])
keep_prob = tf.placeholder(tf.float32)

In [7]:
num_classes = 2
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [8]:
def mlp(x):
    # hidden layer1
    w1 = tf.Variable(tf.random_uniform([784,256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # hidden layer2
    w2 = tf.Variable(tf.random_uniform([256,128]))
    b2 = tf.Variable(tf.zeros([128]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    h2_drop = tf.nn.dropout(h2, keep_prob)
    # output layer
    w3 = tf.Variable(tf.random_uniform([128,2]))
    b3 = tf.Variable(tf.zeros([2]))
    logits= tf.matmul(h2_drop, w3) + b3
    
    return logits

In [9]:
logits = mlp(x)

In [10]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=y))

In [11]:
train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

In [12]:
# initialize
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# train hyperparameters
epoch_cnt = 5
batch_size = int(len(x_train)/10)
iteration = int(len(x_train) // batch_size)

earlystop_threshold = 5
earlystop_cnt = 0

In [13]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    prev_train_acc = 0.0
    max_val_acc = 0.0
    
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op], 
                               feed_dict={x: x_train[start: end], 
                                          y: y_train[start: end], 
                                          keep_prob: 0.9})
            start = batch_size * (i+1) ; end = batch_size * (i+2)
            # Compute train average loss
            avg_loss += loss / iteration
            
        # Validate model
        preds = tf.nn.softmax(logits)  # Apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        # train accuracy
        cur_train_acc = accuracy.eval({x: x_train, y: y_train,keep_prob: 1.0})
        # validation accuarcy
        cur_val_acc = accuracy.eval({x: x_val, y: y_val, keep_prob: 1.0})
        # validation loss
        cur_val_loss = loss_op.eval({x: x_val, y: y_val,keep_prob: 1.0})
        
        print("epoch: "+str(epoch)+
              ", train acc: " + str(cur_train_acc) +
              ", val acc: " + str(cur_val_acc) )
              #', train loss: '+str(avg_loss)+
              #', val loss: '+str(cur_val_loss))
        
        if cur_val_acc < max_val_acc:
            if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                if earlystop_cnt == earlystop_threshold:
                    print("early stopped on "+str(epoch))
                    break
                else:
                    print("overfitting warning: "+str(earlystop_cnt))
                    earlystop_cnt += 1
            else:
                earlystop_cnt = 0
        else:
            earlystop_cnt = 0
            max_val_acc = cur_val_acc
            # Save the variables to file.
            save_path = saver.save(sess, "model/model.ckpt")
        prev_train_acc = cur_train_acc

epoch: 0, train acc: 0.883625, val acc: 0.9
epoch: 1, train acc: 0.8835625, val acc: 0.9
epoch: 2, train acc: 0.8835625, val acc: 0.9
epoch: 3, train acc: 0.8835625, val acc: 0.9
epoch: 4, train acc: 0.8835625, val acc: 0.9


In [14]:
# Start testing
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "model/model.ckpt")
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

INFO:tensorflow:Restoring parameters from model/model.ckpt
[Test Accuracy] : 0.8985
